# Initialization

In [ ]:
# Google Earth Engine packages
import ee
import geemap

# other packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# GIS packages
from pysheds.grid import Grid
import fiona


#Define a function to plot the digital elevation model
def plotFigure(data, label, cmap='Blues'):
    plt.figure(figsize=(12,10))
    plt.imshow(data, extent=grid.extent)
    plt.colorbar(label=label)
    plt.grid()
    

# constants
ee_img = 'ee.image.Image'
ee_ico = 'ee.imagecollection.ImageCollection'

In [ ]:
# initialize GEE at the beginning of session
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()         # authenticate when using GEE for the first time
    ee.Initialize()

In [ ]:
# get config data (will be replaces by config file soon)
filename = 'output/dem_gee.tif'
output_folder = 'output/'
output_gpkg = output_folder + 'catchment_data.gpkg'

dem = [ee_img, 'CGIAR/SRTM90_V4','elevation','SRTM 90m']
#dem = [ee_img, 'USGS/SRTMGL1_003','elevation','SRTM NASA 30m']
#dem = [ee_img', 'MERIT/DEM/v1_0_3','dem', 'MERIT 30m']
#dem = [ee_ico, 'JAXA/ALOS/AW3D30/V3_2','DSM','ALOS DSM']

y, x = 42.300264, 78.091444

# Start GEE and find catchment area

Start with base map

In [ ]:
Map = geemap.Map()
Map

Load selected DEM from GEE catalog and add as layer to map

In [ ]:
if dem[0] == ee_img:
    image = ee.Image(dem[1])
elif dem[0] == ee_ico:
    image = ee.ImageCollection(dem[1]).select(dem[2]).mosaic()
    
srtm_vis = { 'bands': dem[2],
             'min': 0,
             'max': 6000,
            'palette': ['000000', '478FCD', '86C58E', 'AFC35E', '8F7131','B78D4F', 'E2B8A6', 'FFFFFF']
           }
Map.addLayer(image, srtm_vis, dem[3], True, 0.7)

Add configured discharge point to map and automatically draw box with 30km in all directions

In [ ]:
point = ee.Geometry.Point(x,y)
Map.addLayer(point,{'color': 'blue'},'Discharge Point');

box = point.buffer(30000).bounds()
Map.addLayer(box,{'color': 'grey'},'Catchment Area', True, 0.7);
Map.centerObject(box, zoom=9)

Discharge point (marker) and box (polygon/rectangle) can be added manually to the map above. If features have been drawn, they will overrule the configured discharge point and automatically created box.

In [ ]:
for feature in Map.draw_features:
    f_type = feature.getInfo()['geometry']['type']
    if f_type == 'Point':
        point = feature.geometry()
        print("Manually set pouring point will be considered")
    elif f_type == 'Polygon':
        box = feature.geometry()
        print("Manually drawn box will be considered")

Export DEM as .tif file to output folder.

In [ ]:
geemap.ee_export_image(image, filename=filename, scale=30, region=box, file_per_band=False)

# Catchment deliniation

Use <code>pysheds</code> module to determine catchment area for discharge point. The result will be a raster.

In [ ]:
DEM_file = filename
# Plot the DEM
grid = Grid.from_raster(DEM_file)
dem = grid.read_raster(DEM_file)
grid.view(dem)
# Fill depressions in DEM
flooded_dem = grid.fill_depressions(dem)
# Resolve flats in DEM
inflated_dem = grid.resolve_flats(flooded_dem)
# Specify directional mapping
#N    NE    E    SE    S    SW    W    NW
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
# Compute flow directions
fdir = grid.flowdir(inflated_dem, dirmap=dirmap)
#catch = grid.catchment(x=x, y=y, fdir=fdir, dirmap=dirmap, xytype='coordinate')
# Compute accumulation
acc = grid.accumulation(fdir)
# Snap pour point to high accumulation cell
x_snap, y_snap = grid.snap_to_mask(acc > 1000, (x, y))
# Delineate the catchment
catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, xytype='coordinate')
# Clip the DEM to the catchment
grid.clip_to(catch)
clipped_catch = grid.view(catch)

In [ ]:
demView = grid.view(dem, nodata=np.nan)
plotFigure(demView,'Elevation')
plt.show()

Convert catchment raster to polygon and save to output folder as geopackage. 

In [ ]:
from shapely.geometry import Polygon
import pyproj
from shapely.geometry import shape
from shapely.ops import transform

## Create shapefile and save it
shapes = grid.polygonize()

schema = {
    'geometry': 'Polygon',
    'properties': {'LABEL': 'float:16'}
}

catchment_shape = {}
with fiona.open(output_gpkg, 'w',
                #driver='ESRI Shapefile',#
                driver='GPKG',
                layer='catchment_orig',
                crs=grid.crs.srs,
                schema=schema) as c:
    i = 0
    for shape, value in shapes:
        catchment_shape = shape
        rec = {}
        rec['geometry'] = shape
        rec['properties'] = {'LABEL' : str(value)}
        rec['id'] = str(i)
        c.write(rec)
        i += 1      

print(f"Mean catchment elevation is {str(np.nanmean(demView))} m")

Add catchment area to map and calculate area.

In [ ]:
catchment = ee.Geometry.Polygon(catchment_shape['coordinates'])
Map.addLayer(catchment, {}, 'Catchment')

catchment_area = catchment.area().divide(1000*1000).getInfo()
print(f"Catchment area is {catchment_area} km²")

# Determine glaciers in catchment area

Find all glacier that are intersecting catchment area in RGI60 database (for area 13)

In [ ]:
import geopandas as gpd

catchment = gpd.read_file(output_gpkg, layer='catchment_orig')
rgi = gpd.read_file("zip://input/13_rgi60_CentralAsia.zip")

if rgi.crs != catchment.crs:
    print("CRS adjusted")
    rgi = rgi.to_crs(catchment.crs)

rgi_catchment = gpd.sjoin(rgi,catchment,how='inner',predicate='intersects')

Some glaciers do not belong to catchment but are intersecting the derived catchment area. Therefore, the percentage of the glacier will be calculated to determine whether glacier will be part of catchment or not (>=50% of its area needs to be in catchment). Glaciers outside catchment with overlapping area will reduce catchment area.
Results for each glacier can be printed if needed.

In [ ]:
# intersects selects too many. calculate percentage of glacier area that is within catchment
rgi_catchment['rgi_area'] = rgi_catchment.area

gdf_joined = gpd.overlay(catchment,rgi_catchment, how='union')
gdf_joined['area_joined'] = gdf_joined.area
gdf_joined['share_of_area'] = (gdf_joined['area_joined'] / gdf_joined['rgi_area'] * 100)

results = (gdf_joined
           .groupby(['RGIId','LABEL_1'])
           .agg({'share_of_area':'sum'}))

#print(results.sort_values(['share_of_area'],ascending=False))

In [ ]:
rgi_catchment = pd.merge(rgi_catchment, results, on="RGIId")
rgi_in_catchment = rgi_catchment.loc[rgi_catchment['share_of_area'] >= 50]
rgi_out_catchment = rgi_catchment.loc[rgi_catchment['share_of_area'] < 50]

catchment_new = gpd.overlay(catchment, rgi_out_catchment, how='difference')
catchment_new = gpd.overlay(catchment_new, rgi_in_catchment, how='union')
catchment_new = catchment_new.dissolve()[['LABEL_1','geometry']]

Export data to existing geopackage:
<ul>
    <li>RGI glaciers within catchment</li>
    <li>RGI glaciers outside catchment</li>
    <li>Adjusted catchment area based in RGI glaciers</li>
</ul>

In [ ]:
rgi_in_catchment.to_file(output_gpkg, layer='rgi_in', driver='GPKG')
rgi_out_catchment.to_file(output_gpkg, layer='rgi_out', driver='GPKG')
catchment_new.to_file(output_gpkg, layer='catchment_new', driver='GPKG')

Add determined glaciers and new catchment area to map.

In [ ]:
c_new = geemap.geopandas_to_ee(catchment_new)
Map.addLayer(c_new, {'color': 'orange'}, "Catchment New")

rgi = geemap.geopandas_to_ee(rgi_in_catchment)
Map.addLayer(rgi, {'color': 'white'}, "RGI60")

The thickness of each glacier must be determined from raster files. Depending on the RGI IDs that are within catchment area, the corresponding raster files will be downloaded from server and stored in output folder 

In [ ]:
rgiids = rgi_in_catchment['RGIId'].sort_values()

areas = []
req_file_names = []
for rgiid in rgiids:
    # prepare filename that will be checked in *.zip archive in the next step
    req_file_names.append(f'{rgiid}_thickness.tif')
    area = rgiid.split('.')[0]
    if area not in areas: areas.append(area)

In [ ]:
import requests
from zipfile import ZipFile
import io

cnt = 0
for area in areas:
    url = f'https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/315707/composite_thickness_RGI60-{area}.zip?sequence=15&isAllowed=y'

    r = requests.get(url, stream=True)

    with ZipFile(io.BytesIO(r.content), 'r') as zipObj:
        # Get a list of all archived file names from the zip
        listOfFileNames = zipObj.namelist()
        for fileName in listOfFileNames:
            file = fileName.split('/')[1]
            if file in req_file_names:
                cnt += 1
                zipObj.extract(fileName, output_folder+'RGI')
print(f'{cnt} files have been extracted')